# Needleman–Wunsch algorithm implementation- HOANG Hai Nam

## Pre-requisites ##

I'm sorry that I couldn't make the implementation of the substitution matrix better, the packages and documentations I've found on how to do them are very obscure and confusing, Bio.align.substitution matrix module seems like a better solution but the matrix would be imported as a mixed array that I could not seem to manipulate with numpy's functions, as such the deprecated Bio.SubsMat package was used, as it generate the exact dictionary seen below, I've gone and copied the dictionary in case the package was removed from Biopython. Even if the package wasn't removed I could not figure out a way to call specific matrices from an input as the function to do so isn't very modular.

In [1]:
import numpy as np
from Bio.SubsMat import MatrixInfo
import math as mt
import secrets as sc
# SUBMAT = MatrixInfo.blosum62 ## soon to be deprecated, bio.align.substitution is stupid and
#                              ## there's no easy way to get a dictionary., using matrix.info isnt exactly modular either..
                                ## both sucks..
## back up of 62(from matrixinfo), in case they actually nuked the thing, sorry for the super long dictionary :(#
blosum62 = {('W', 'F'): 1, ('L', 'R'): -2, ('S', 'P'): -1, ('V', 'T'): 0, ('Q', 'Q'): 5, ('N', 'A'): -2, ('Z', 'Y'): -2,
            ('W', 'R'): -3, ('Q', 'A'): -1, ('S', 'D'): 0, ('H', 'H'): 8, ('S', 'H'): -1, ('H', 'D'): -1,
            ('L', 'N'): -3, ('W', 'A'): -3, ('Y', 'M'): -1, ('G', 'R'): -2, ('Y', 'I'): -1, ('Y', 'E'): -2,
            ('B', 'Y'): -3, ('Y', 'A'): -2, ('V', 'D'): -3, ('B', 'S'): 0, ('Y', 'Y'): 7, ('G', 'N'): 0, ('E', 'C'): -4,
            ('Y', 'Q'): -1, ('Z', 'Z'): 4, ('V', 'A'): 0, ('C', 'C'): 9, ('M', 'R'): -1, ('V', 'E'): -2, ('T', 'N'): 0,
            ('P', 'P'): 7, ('V', 'I'): 3, ('V', 'S'): -2, ('Z', 'P'): -1, ('V', 'M'): 1, ('T', 'F'): -2, ('V', 'Q'): -2,
            ('K', 'K'): 5, ('P', 'D'): -1, ('I', 'H'): -3, ('I', 'D'): -3, ('T', 'R'): -1, ('P', 'L'): -3,
            ('K', 'G'): -2, ('M', 'N'): -2, ('P', 'H'): -2, ('F', 'Q'): -3, ('Z', 'G'): -2, ('X', 'L'): -1,
            ('T', 'M'): -1, ('Z', 'C'): -3, ('X', 'H'): -1, ('D', 'R'): -2, ('B', 'W'): -4, ('X', 'D'): -1,
            ('Z', 'K'): 1, ('F', 'A'): -2, ('Z', 'W'): -3, ('F', 'E'): -3, ('D', 'N'): 1, ('B', 'K'): 0, ('X', 'X'): -1,
            ('F', 'I'): 0, ('B', 'G'): -1, ('X', 'T'): 0, ('F', 'M'): 0, ('B', 'C'): -3, ('Z', 'I'): -3, ('Z', 'V'): -2,
            ('S', 'S'): 4, ('L', 'Q'): -2, ('W', 'E'): -3, ('Q', 'R'): 1, ('N', 'N'): 6, ('W', 'M'): -1, ('Q', 'C'): -3,
            ('W', 'I'): -3, ('S', 'C'): -1, ('L', 'A'): -1, ('S', 'G'): 0, ('L', 'E'): -3, ('W', 'Q'): -2,
            ('H', 'G'): -2, ('S', 'K'): 0, ('Q', 'N'): 0, ('N', 'R'): 0, ('H', 'C'): -3, ('Y', 'N'): -2, ('G', 'Q'): -2,
            ('Y', 'F'): 3, ('C', 'A'): 0, ('V', 'L'): 1, ('G', 'E'): -2, ('G', 'A'): 0, ('K', 'R'): 2, ('E', 'D'): 2,
            ('Y', 'R'): -2, ('M', 'Q'): 0, ('T', 'I'): -1, ('C', 'D'): -3, ('V', 'F'): -1, ('T', 'A'): 0,
            ('T', 'P'): -1, ('B', 'P'): -2, ('T', 'E'): -1, ('V', 'N'): -3, ('P', 'G'): -2, ('M', 'A'): -1,
            ('K', 'H'): -1, ('V', 'R'): -3, ('P', 'C'): -3, ('M', 'E'): -2, ('K', 'L'): -2, ('V', 'V'): 4,
            ('M', 'I'): 1, ('T', 'Q'): -1, ('I', 'G'): -4, ('P', 'K'): -1, ('M', 'M'): 5, ('K', 'D'): -1,
            ('I', 'C'): -1, ('Z', 'D'): 1, ('F', 'R'): -3, ('X', 'K'): -1, ('Q', 'D'): 0, ('X', 'G'): -1,
            ('Z', 'L'): -3, ('X', 'C'): -2, ('Z', 'H'): 0, ('B', 'L'): -4, ('B', 'H'): 0, ('F', 'F'): 6, ('X', 'W'): -2,
            ('B', 'D'): 4, ('D', 'A'): -2, ('S', 'L'): -2, ('X', 'S'): 0, ('F', 'N'): -3, ('S', 'R'): -1,
            ('W', 'D'): -4, ('V', 'Y'): -1, ('W', 'L'): -2, ('H', 'R'): 0, ('W', 'H'): -2, ('H', 'N'): 1,
            ('W', 'T'): -2, ('T', 'T'): 5, ('S', 'F'): -2, ('W', 'P'): -4, ('L', 'D'): -4, ('B', 'I'): -3,
            ('L', 'H'): -3, ('S', 'N'): 1, ('B', 'T'): -1, ('L', 'L'): 4, ('Y', 'K'): -2, ('E', 'Q'): 2, ('Y', 'G'): -3,
            ('Z', 'S'): 0, ('Y', 'C'): -2, ('G', 'D'): -1, ('B', 'V'): -3, ('E', 'A'): -1, ('Y', 'W'): 2, ('E', 'E'): 5,
            ('Y', 'S'): -2, ('C', 'N'): -3, ('V', 'C'): -1, ('T', 'H'): -2, ('P', 'R'): -2, ('V', 'G'): -3,
            ('T', 'L'): -1, ('V', 'K'): -2, ('K', 'Q'): 1, ('R', 'A'): -1, ('I', 'R'): -3, ('T', 'D'): -1,
            ('P', 'F'): -4, ('I', 'N'): -3, ('K', 'I'): -3, ('M', 'D'): -3, ('V', 'W'): -3, ('W', 'W'): 11,
            ('M', 'H'): -2, ('P', 'N'): -2, ('K', 'A'): -1, ('M', 'L'): 2, ('K', 'E'): 1, ('Z', 'E'): 4, ('X', 'N'): -1,
            ('Z', 'A'): -1, ('Z', 'M'): -1, ('X', 'F'): -1, ('K', 'C'): -3, ('B', 'Q'): 0, ('X', 'B'): -1,
            ('B', 'M'): -3, ('F', 'C'): -2, ('Z', 'Q'): 3, ('X', 'Z'): -1, ('F', 'G'): -3, ('B', 'E'): 1,
            ('X', 'V'): -1, ('F', 'K'): -3, ('B', 'A'): -2, ('X', 'R'): -1, ('D', 'D'): 6, ('W', 'G'): -2,
            ('Z', 'F'): -3, ('S', 'Q'): 0, ('W', 'C'): -2, ('W', 'K'): -3, ('H', 'Q'): 0, ('L', 'C'): -1,
            ('W', 'N'): -4, ('S', 'A'): 1, ('L', 'G'): -4, ('W', 'S'): -3, ('S', 'E'): 0, ('H', 'E'): 0, ('S', 'I'): -2,
            ('H', 'A'): -2, ('S', 'M'): -1, ('Y', 'L'): -1, ('Y', 'H'): 2, ('Y', 'D'): -3, ('E', 'R'): 0,
            ('X', 'P'): -2, ('G', 'G'): 6, ('G', 'C'): -3, ('E', 'N'): 0, ('Y', 'T'): -2, ('Y', 'P'): -3,
            ('T', 'K'): -1, ('A', 'A'): 4, ('P', 'Q'): -1, ('T', 'C'): -1, ('V', 'H'): -3, ('T', 'G'): -2,
            ('I', 'Q'): -3, ('Z', 'T'): -1, ('C', 'R'): -3, ('V', 'P'): -2, ('P', 'E'): -1, ('M', 'C'): -1,
            ('K', 'N'): 0, ('I', 'I'): 4, ('P', 'A'): -1, ('M', 'G'): -3, ('T', 'S'): 1, ('I', 'E'): -3, ('P', 'M'): -2,
            ('M', 'K'): -1, ('I', 'A'): -1, ('P', 'I'): -3, ('R', 'R'): 5, ('X', 'M'): -1, ('L', 'I'): 2,
            ('X', 'I'): -1, ('Z', 'B'): 1, ('X', 'E'): -1, ('Z', 'N'): 0, ('X', 'A'): 0, ('B', 'R'): -1, ('B', 'N'): 3,
            ('F', 'D'): -3, ('X', 'Y'): -1, ('Z', 'R'): 0, ('F', 'H'): -1, ('B', 'F'): -3, ('F', 'L'): 0,
            ('X', 'Q'): -1, ('B', 'B'): 4}

C:\Users\Rae\anaconda3\lib\site-packages\Bio\SubsMat\__init__.py:126: BiopythonDeprecationWarning: Bio.SubsMat has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.substitution_matrices as a replacement, and contact the Biopython developers if you still need the Bio.SubsMat module.
  warnings.warn(


The dictionary contains elements as tuples of 2 amino acids, and keys being substitution values to them. For example, to look up the substitution value of P and M match, the call for value would be blosum62.get(('P', 'M')). As explained in the code, I could not find a way to implement a substitution matrix dictionary from Bio.Align.substitution_matrices, therefore I used the Bio.SubsMat module and saved the resulting dictionary as a backup in case that module gets removed from Biopython. Using Bio.SubsMat, I couldn't call for a matrix using a variable so it's not very modular as functions go.

### Inputs ###

In [2]:
SUBMAT = blosum62
GAPPEN = -7
SEQ1 = "MGHFTEEDKATITSLWGKVNVEDAGGETLGRLLVVYPWTQRFFDSFGNLSSASAIMGNPKVKAHGKKVLTSLGDAIKHLDDLKGTFAQLSELHCDKLHVDPENFKLLGNVLVTVLAIHFGKEFTPEVQASWQKMVTGVASALSSRYH"
SEQ2 = "MVLSGEDKSNIKAAWGKIGGHGAEYGAEALERMFASFPTTKTYFPHFDVSHGSAQVKGHGKKVADALASAAGHLDDLPGALSALSDLHAHKLRVDPVNFKLLSHCLLVTLASHHPADFTPAVHASLDKFLASVSTVLTSKYR"
#both sequences were taken from Uniprot
#seq1: P69892 (HBG2_HUMAN)
#seq2: P01942 (HBA_MOUSE)

## Building the Matrix ##

In [3]:
def matstart(seq1, seq2, g):
    '''Builds the scoring matrix, seq1 and seq2 being the 2 sequences, g is the gap penalty'''
    seq1 = list(seq1)
    seq2 = list(seq2)
    mat = np.zeros([len(seq1) + 1, len(seq2) + 1])  # makeshift python matrix would also work here instead of numpy
    seq1.insert(0, 0)                               ## but im lazy and this is the easiest way to get an empty matrix
    seq2.insert(0, 0)
    for i in range(0, len(mat[0][:])):  # First penalty row gen
        mat[0][i] = i * g
    for i in range(0, len(mat)):  # First penalty col gen
        mat[i][0] = i * g
    for r in range(1, len(mat)):# loops row to row 
        for c in range(1, len(mat[0][:])): #loops column to column
            key = SUBMAT.get(tuple([seq1[r], seq2[c]]))
            if key is None:  ##SUBMAT only prints in 1 direction, can get None if it's reversed
                key = SUBMAT.get(tuple([seq2[c], seq1[r]])) #reversal of getkey
            mat[r][c] = max(mat[r - 1][c] + g, mat[r][c - 1] + g, mat[r - 1][c - 1] + key)
    return mat


This function builds the scoring matrix that will be used by the walkback function. First the first row and collumn scores are calculated as being simply multiplications of the gap penalty. Then from left to right and gradually going down, score for each cell is calculated and added to the matrix until all cells are filled, the score are calculated using the neighboring 3 cells(above, left and diagonally top left), as well as the use of the substitution matrix in the case of the diagonal neighbor.

In [4]:
MAT = matstart(SEQ1, SEQ2, GAPPEN)
print(MAT)

[[    0.    -7.   -14. ...  -980.  -987.  -994.]
 [   -7.     5.    -2. ...  -968.  -975.  -982.]
 [  -14.    -2.     2. ...  -955.  -962.  -969.]
 ...
 [-1015. -1003.  -992. ...   252.   245.   241.]
 [-1022. -1010.  -999. ...   245.   259.   252.]
 [-1029. -1017. -1006. ...   238.   252.   259.]]


## Walkback function ##

In [5]:
def walkback(mat, seq1, seq2, g):
    '''Walks back from bottom right of the matrix to the bottom left, finding the possible route, at each junction where there are multiple
    possible paths, a random route is chosen with secret.choice()
    seq1 and seq2 are the alignment sequences, mat is the provided matrix and g is the gap penalty'''
    seq1 = list(seq1)
    seq2 = list(seq2)
    seq1.insert(0, 0)
    seq2.insert(0, 0)
    r = len(mat) - 1  # starting point at the bottom 
    c = len(mat[0][:]) - 1 #starting point at the far right
    seq1f = []
    seq2f = []
    score = mat[r][c] #starting score at bottom right coordinates
    while r > 0 and c > 0:
        possi = []
        key = SUBMAT.get(tuple([seq1[r], seq2[c]]))
        if key is None:  # SUBMAT only prints in 1 direction, can get None if it's reversed
            key = SUBMAT.get(tuple([seq2[c], seq1[r]]))
        if mat[r][c - 1] + g == mat[r][c]:
            possi.append('W')
        if mat[r - 1][c] + g == mat[r][c]:
            possi.append('N')
        if mat[r - 1][c - 1] + key == mat[r][c]:
            possi.append('NW')
        prime = sc.choice(possi) # random choice between possibilities
        if prime == 'W':  # choice of skipping 1 on seq1, go left/west in the matrix
            seq1f.append('-')
            seq2f.append(seq2[c])
            c -= 1
        elif prime == 'N':  # choice of skipping 1 on seq 2, go up/north in the matrix
            seq1f.append(seq1[r])
            seq2f.append('-')
            r -= 1
        elif prime == 'NW':  # perfect match, go up and left(northwest)
            seq1f.append(seq1[r])
            seq2f.append(seq2[c])
            c -= 1
            r -= 1
        score += mat[r][c] # updates the score after step is taken
    if r == 0 and c > r:  # in the case that there's a tail for seq1
        seq1f.append((c - r) * '-')
        while c > 0:
            seq2f.append(seq2[c])
            c -= 1
            score += mat[r][c]
    elif c == 0 and r > c:  # in the case that there's a tail for seq1
        seq2f.append((r - c) * '-')
        while r > 0:
            seq1f.append(seq1[r])
            r -= 1
            score += mat[r][c]
    seq1f.reverse()
    seq2f.reverse()
    seq1f = ''.join(seq1f)
    seq2f = ''.join(seq2f)
    return (seq1f, seq2f, score)


This is the walkback function, it uses r and c (row and collumn) as coordinates and starts from the bottom right, each time it's looped it calculates the possible ways it can head up(r-1), left(c-1) or diagonally(r-1)(c-1) using the value of the current coordinate cell and possible backtrack value. There're a maximum of 3 possibilities each junction and based on the random choice function it picks one random route. Each time a move is chosen, both final sequence lists are added one amino acid or a dash(-) if a skip/gap was preferred. These 2 lists are then reveresed and turned into strings after all loops has been done and the current cell has reached (r=0, c=0). The output of the function is a 3-tuple of the first sequence and second sequence with gaps, and the score. The score is calculated by the sum of cell values in the route. Originally I wanted to do the random choice at junction based on a probability distribution calculated from the local score, however, I could not treat the score in a fair manner due to the possibility of having a negative local score, normalising such score and deriving a probability would possibly introduce biases against or for gaps formation(which tend to introduce a negative local score). Not to mention deciding what is best locally is rather hard as this isn't usually done, most documentation on Needleman Wunsch aligners use branching tree recursivity programming to reach all possible alignments.

## Random Sampling?

In [6]:
def randomsampler(mat, seq1, seq2, g, N):  # AKA im too stupid to implement 3 way branching recursivity :(
    '''Relies on the walkback function. Accepts matrix, 1st sequence, 2nd sequence, penalty gap and numbers of iterations
    Essentially loops for N number of times, sorts the pool by score value and returns only THE best
    scoring matrices, which means there's no threshold, these matrices have THE top score, 
    returns a list of these matrices. mat is the matrix, seq1 and seq2 are alignment sequences, g is the gap penalty and N is the number of loops
    Prints have been disabled here because i made a very cool function ahead that does just that, but a lot better :)'''
    pool = []
    for i in range(0, N):
        if walkback(mat, seq1, seq2, g) not in pool:
            pool.append(walkback(mat, seq1, seq2, g))
    pool = sorted(pool, key=lambda x: x[2], reverse=True)# sorting the pool by score, descending
    pooltop = []
    for pair in pool:
        if pair[2] == pool[0][2]: #filtration of non TOP scoring tuples
            pooltop.append(pair)
    #score = pooltop[0][2]
    #for pair in pooltop:
        #print(pair[0])
        #print(len(pair[0]) * '|')
        #print(pair[1])
        #print("Score = ", pair[2])
        #print('\n')
    return pooltop

In [7]:
PAIRPOOL = randomsampler(MAT, SEQ1, SEQ2, GAPPEN, 100)

In my testings, for these 2 sequences of 147/142 length, most(if not all) possibilities of matches(bad or good score) are achieved between N = 1000 and N = 10000. After this, the amount of possible matches don't change(I even tested with N = 1M and it took my machine 7 minutes to complete, do not do this unless you have a very fast CPU). Since it is possible for their to be multiple valid movements(gap/match placement) at junctions, and the final score can differ, and there is seemingly no way to tell which way is optimal for the final, global score, a random sampling function is implemented here. This function will repeat the walkback function multiple times, and since the choice at junctions are random within the walkback function, it is possible to obtain most if not all possible alignments. The second half of this function sorts the pool of alignments by score, and picks only the highest scoring ones, as occasionally there can be multiple, different alignments with the same (top) score. 

## Display of results

In [8]:
def display(pool, chunklength=50):
    '''Takes in a pool/list of 3-tuples and display them like they do in the uniprot/fasta thingpool being a list of 3-tuples
    containing alignments( of 2 sequences) and their scores, chunklength is the length that the alignments are broken into
    this is stupidly over-engineered unintuitive code but i spent too much time on it so I'm throwing this in :')
    also a tiny bit proud of it..'''
    for pair in pool:
        chkseq1 = []
        chkseq2 = []
        connectors = []
        conchk =[]
        for r in range(len(pair[0])):
            if pair[0][r] == '-' or pair[1][r] == '-':
                connectors.append(" ")
            else:
                connectors.append("|")
        for i in range(0, len(pair[0]), chunklength): #breaks 2 sequences in the pool and the connectors into chunks of "chunklength"
            chkseq1.append(pair[0][i:chunklength + i])
            chkseq2.append(pair[1][i:chunklength + i])
            conchk.append(connectors[i:chunklength + i])
        for i in range(len(chkseq1)):
            print((i * chunklength + 1),
                  ((len(chkseq1[i])-len(str((i * len(chkseq1[i]) + 1)))-len(str((i * len(chkseq1[i]) + 1 + len(chkseq1[i])))))-1) * ' ',
                  (i * chunklength + len(chkseq1[i])) )## I don't even know how this worked... but glad it did!
            print(chkseq1[i])                           ## there're 3 things being printed here, 1st pos, mid spaces
            print(''.join(conchk[i]))                ## and last pos(of the chunk), number of spaces is calculated
            print(chkseq2[i])                           #using length of the current chunk, length of the digits of the index
        print("Score = ", pair[2])                      # this is stupid code and#unintuitive as hell... 
        print('\n')                                     # i hate how much time i spent on this..

This is the display funciton that splits alignments into chunks of defined length, there's not much to say except for the quite unintuitive calculation of numbers of spaces between the position markers. The number of spaces are calculated based on the position marker's number of digits, so that the end markers align with the actual aminoacid. The connectors can better show gaps between the alignments.

In [9]:
display(PAIRPOOL, 50)

1                                                50
MGHFTEEDKATITSLWGKVNVEDA--GGETLGRLLVVYPWTQRFFDSFGN
|| |||||||||||||||||||||  |||||||||||||||||||||| |
MV-LSGEDKSNIKAAWGKIGGHGAEYGAEALERMFASFPTTKTYFPHF-D
51                                              100
LSSASAIMGNPKVKAHGKKVLTSLGDAIKHLDDLKGTFAQLSELHCDKLH
||||||     |||||||||||||||||||||||||||||||||||||||
VSHGSA-----QVKGHGKKVADALASAAGHLDDLPGALSALSDLHAHKLR
101                                             149
VDPENFKLLGNVLVTVLAIHFGKEFTPEVQASWQKMVTGVASALSSRYH
|||||||||||||||||||||||||||||||||||||||||||||||||
VDPVNFKLLSHCLLVTLASHHPADFTPAVHASLDKFLASVSTVLTSKYR
Score =  16395.0




Here the sequences are split into chunks of length(50) for easier viewing, if you want to view the entire alignment on the same lign, run the display code below.

In [10]:
display(PAIRPOOL, len(PAIRPOOL[0][0])) 

1                                                                                                                                                  149
MGHFTEEDKATITSLWGKVNVEDA--GGETLGRLLVVYPWTQRFFDSFGNLSSASAIMGNPKVKAHGKKVLTSLGDAIKHLDDLKGTFAQLSELHCDKLHVDPENFKLLGNVLVTVLAIHFGKEFTPEVQASWQKMVTGVASALSSRYH
|| |||||||||||||||||||||  |||||||||||||||||||||| |||||||     ||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
MV-LSGEDKSNIKAAWGKIGGHGAEYGAEALERMFASFPTTKTYFPHF-DVSHGSA-----QVKGHGKKVADALASAAGHLDDLPGALSALSDLHAHKLRVDPVNFKLLSHCLLVTLASHHPADFTPAVHASLDKFLASVSTVLTSKYR
Score =  16395.0




## Significance calculation

In [11]:
def scorepool(len1, len2, g, N, amount):
    '''Generates a certain number of pairs of amino acids, each pair is fed into the random sampler function, have the top score taken and then return a score pool as a list
    len1 and len2 being the 2 lengths, g is the gap penalty, N is the number of loops for the random sampler, amount being the numbers of pairs generated.
    WARNING: amount scales up N in randomsampler() function, be careful with amount here, higher values scale total processing exponentially!!!'''
    pool = []
    AAs = 'ARNDCEQGHILKMFPSTWYV'
    scpool = []
    for i in range(0, amount):
        seq1 = []
        seq2 = []
        for elem in range(len1):
            seq1 += sc.choice(AAs)
        for elem in range(len2):
            seq2 += sc.choice(AAs)
        pool.append((seq1, seq2))
    for pair in pool:
        pairmat = matstart(pair[0], pair[1], g)
        result = randomsampler(pairmat, pair[0], pair[1], g, N)
        scpool.append(result[0][2])
    return scpool

The scorepool() function takes in as lengths of the aligning sequences, gap penalty and number of pairs and returns a list of the (top) score of their alignments through randomsampler() function. Random sequences are generated using sc.choice for each amino acid. Ideally these pairs would have the same sequence lengths as the original alignment for significance score calculation and having a high amount of pairs/alignments generated for more interesting Z-score. 

In [12]:
Z_POOL = scorepool(len(SEQ1), len(SEQ2), GAPPEN, 100, 45) # generation of a pool of scores obtained from alignment of random sequences

In [13]:
def significance(score, pool):
    '''takes in a score and calculate its significance when put into the total pool'''
    newpool = list(pool)
    stdv = 0
    mean = score - (sum(newpool)/len(newpool))
    for s in newpool:
        stdv += (s - mean)**2
    stdv = mt.sqrt(stdv/len(newpool))
    Z = (score - mean)/stdv
    return Z

This significance function calculates the significance(Z-score) of a score against a generated pool of scores. Using simple calculations of sum, standard deviation of the distribution. Will calculate the score as if it was part of the pool, but will not change the pool after it is ran! This is assuming score is NOT a part of the pool originally.

In [14]:
significance(PAIRPOOL[0][2], Z_POOL)

-0.19370362210202144

To regenerate a new pool of scores, rerun Z_POOL